In [1]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
import numpy
import copy
import pyzx as zx
import qiskit
import qiskit.providers.aer.noise as noise
import ipyparallel as ipp
from qiskit.providers.aer import AerSimulator
from openfermion.linalg import givens_decomposition_square as givensRotns
from qiskit.opflow import I,X,Y,Z,StateFn,AbelianGrouper
from qiskit_nature.mappers.second_quantization import JordanWignerMapper
from qiskit_nature.problems.second_quantization.electronic.builders import fermionic_op_builder
from qiskit.quantum_info import Pauli
from qiskit.chemistry import FermionicOperator
from qiskit.aqua.operators.legacy import op_converter
from openfermion.circuits import slater_determinant_preparation_circuit
from qiskit import QuantumCircuit,execute,QuantumRegister,ClassicalRegister
from qiskit import Aer
from joblib import Parallel,delayed
from qiskit.aqua.operators import WeightedPauliOperator
from qiskit.chemistry.components.variational_forms import UCCSD
import scipy
import time
import numba
from numba import jit

In [2]:
c =ipp.Client()
num_qubits=8
v = c.load_balanced_view([0,1,2,3,4,5,6,7])

TimeoutError: Hub connection request timed out

In [ ]:
#Construct Model Hamiltonian
def countYgates(pauli_label):
    countYgates = sum(map(lambda x : 1 if 'Y' in x else 0, pauli_label))
    return countYgates
def qubitOp(h1,h2):
    qubit_op=FermionicOperator(h1,h2).mapping('jordan_wigner')
    return qubit_op
def egBandHamiltonian(U):
    #Getting chemical Potential for Half-Filling
    with open('../chem_pot_for_Half_Fill.txt','r') as f:
        lines=f.readlines()[1:]
        for line in lines:
            elems=line.split()
            if int(elems[0])==U:
                muHalf=float(elems[1]) #Chem Pot for a given Hubbard U
    #Getting the one body and two body interaction vertexes
    with open('../'+str(U)+'/v1e.dat','r') as f:
            lines=f.readlines()[1:]
            num_sites=4
            chem_pot=numpy.zeros((2*num_sites,2*num_sites))
            eg_h1=numpy.zeros((2*num_sites,2*num_sites))
            for line in lines:
                elems=line.split()
                eg_h1[int(elems[0])][int(elems[1])]=float(elems[2])
                eg_h1[int(elems[0])+num_sites][int(elems[1])+num_sites]=float(elems[2])
            for i in range(2*num_sites):
                chem_pot[i][i]=-muHalf
            eg_h1=eg_h1+chem_pot       
    with open('../'+str(U)+'/v2e.dat','r') as f:
        num_sites=4
        eg_h2=numpy.zeros((2*num_sites,2*num_sites,2*num_sites,2*num_sites))
        for line in f:
            if "#" in line:
                continue
            line = line.split()
            i,j,k,l = map(int, line[:4])
            val = float(line[4])
            eg_h2[i,j,k,l] = eg_h2[i+num_sites,j+num_sites,k,l] = eg_h2[i,j,k+num_sites,l+num_sites]             = eg_h2[i+num_sites,j+num_sites,k+num_sites,l+num_sites] = 0.5*val  # convention with 0.5 factor included.
    qubitH=qubitOp(eg_h1,eg_h2)
    #qubitH=Z2Symmetries.two_qubit_reduction(qubitH, 4)
    Hmat=op_converter.to_matrix_operator(qubitH).dense_matrix
    w,v=numpy.linalg.eigh(Hmat)
    Eg=w[0]
    state_g=v[:,0]
    return qubitH,Hmat,Eg,state_g

In [ ]:
#Energy estimate statevector
def Energy(params,save_steps=False):
    circ=var_form_base.construct_circuit(parameters=params)
    state=execute(circ,backend,shots=1024).result().get_statevector() 
    E=(numpy.conjugate(state)@Hmat@state).real
    if save_steps==True:
        with open('paramsForQubitAdapt_eg_model_JW.txt','+a') as f:
            Str=["{:0.16f}".format(params[i].real) for i in range(len(params))]
            print('['+','.join(Str)+']'+'#'+"{:0.16f}".format(E.real),file=f)
    return E

In [1]:
#Energy estimate shots
def egBandHamiltonianPartition(U):
    #Getting chemical Potential for Half-Filling
    with open('../chem_pot_for_Half_Fill.txt','r') as f:
        lines=f.readlines()[1:]
        for line in lines:
            elems=line.split()
            if int(elems[0])==U:
                muHalf=float(elems[1]) #Chem Pot for a given Hubbard U
    #Getting the one body and two body interaction vertexes
    with open('../'+str(U)+'/v1e.dat','r') as f:
            lines=f.readlines()[1:]
            num_sites=4
            chem_pot=numpy.zeros((2*num_sites,2*num_sites))
            eg_h1_0=numpy.zeros((2*num_sites,2*num_sites))
            eg_h1_1=numpy.zeros((2*num_sites,2*num_sites))
            for line in lines:
                line=line.split()
                i,j=map(int,line[:2])
                val=float(line[2])
                if(i==j):
                    eg_h1_0[i,j]=eg_h1_0[i+num_sites,j+num_sites]=val
                else:
                    eg_h1_1[i,j]=eg_h1_1[i+num_sites,j+num_sites]=val
            for i in range(2*num_sites):
                chem_pot[i][i]=-muHalf
            eg_h1_0=eg_h1_0+chem_pot       
    with open('../'+str(U)+'/v2e.dat','r') as f:
        num_sites=4
        eg_h2_0=numpy.zeros((2*num_sites,2*num_sites,2*num_sites,2*num_sites))
        eg_h2_1=numpy.zeros((2*num_sites,2*num_sites,2*num_sites,2*num_sites))
        for line in f:
            if "#" in line:
                continue
            line = line.split()
            i,j,k,l = map(int, line[:4])
            val = float(line[4])
            if((i==j) and (k==l)):
                eg_h2_0[i,j,k,l] = eg_h2_0[i+num_sites,j+num_sites,k,l] = eg_h2_0[i,j,k+num_sites,l+num_sites] \
                = eg_h2_0[i+num_sites,j+num_sites,k+num_sites,l+num_sites] = 0.5*val  # convention with 0.5 factor included.
            else:
                eg_h2_0[i,j,k,l] = eg_h2_0[i+num_sites,j+num_sites,k+num_sites,l+num_sites] = 0.5*val  # convention with 0.5 factor included.
                eg_h2_1[i+num_sites,j+num_sites,k,l] = eg_h2_1[i,j,k+num_sites,l+num_sites] = 0.5*val
    return eg_h1_0,eg_h1_1,eg_h2_0,eg_h2_1
def qubitHamiltonianGroups(U):
    def qubitOp(h1,h2):
        fer_op=fermionic_op_builder.build_ferm_op_from_ints(h1,h2)
        mapper=JordanWignerMapper()
        qubit_op=mapper.map(fer_op)
        return qubit_op
    eg_h1_0,eg_h1_1,eg_h2_0,eg_h2_1=egBandHamiltonianPartition(U)
    #grouping-1
    qubitH_0=qubitOp(eg_h1_1,numpy.zeros((8,8,8,8)))
    qubitH_1=qubitOp(eg_h1_0,eg_h2_0)
    qubitH_2=qubitOp(numpy.zeros((8,8)),eg_h2_1)
    return qubitH_0,qubitH_1,qubitH_2
def Energy_Estimator(params,num_shots_arr=[2**16,2**16,2**16],basis_gates=None,noise_model=None):
    def qubitOp(h1,h2):
        fer_op=fermionic_op_builder.build_ferm_op_from_ints(h1,h2)
        mapper=JordanWignerMapper()
        qubit_op=mapper.map(fer_op)
        return qubit_op
    def OffDiag1BodyTerms(Op):
        h1=eg_h1_1#+eg_h1_0
    #     Op=qubitOp(h1,numpy.zeros((8,8,8,8)))
        Op=Op.reduce()
        w,v=numpy.linalg.eigh(h1)
        givens=givensRotns(v.T)[0]
        OpNew=qubitOp(numpy.diag(w),numpy.zeros((8,8,8,8)))
        #Prepare gate implementing Givens Rotn for appending to ansatz circ pre-measurement
        circ=QuantumCircuit(8)
        for rots in givens:
            for tup in rots:
                circ.cnot(tup[1],tup[0])
                circ.cry(-2*tup[2],tup[0], tup[1])
                circ.cnot(tup[1], tup[0])
                circ.rz(tup[3],tup[1])     
        circ0_8gate=circ.to_instruction()
        return circ0_8gate,OpNew
    def OffDiag2BodyTerms(Op):
        theta=numpy.pi/4
        #Prepare UnitaryOp for rotating Hamilotnian terms
        O_0_1_up=(numpy.cos(theta/2.)*I^I^I^I^I^I^I^I)+(1j*numpy.sin(theta/2.)*I^I^I^I^I^I^X^Y)
        O_1_0_up=(numpy.cos(theta/2.)*I^I^I^I^I^I^I^I)+(-1j*numpy.sin(theta/2.)*I^I^I^I^I^I^Y^X)
        O_0_1_down=(numpy.cos(theta/2.)*I^I^I^I^I^I^I^I)+(1j*numpy.sin(theta/2.)*I^I^X^Y^I^I^I^I)
        O_1_0_down=(numpy.cos(theta/2.)*I^I^I^I^I^I^I^I)+(-1j*numpy.sin(theta/2.)*I^I^Y^X^I^I^I^I)
        O_01=O_0_1_up@O_1_0_up@O_0_1_down@O_1_0_down
        O=O_01.reduce()
        Odag=O.adjoint()
        rotH=O@Op@Odag
        rotH=rotH.reduce().reduce()
         #theta=numpy.pi/4
        #Prepare gate implementing Givens Rotn for appending to ansatz circ pre-measurement
        RotMat=(1/numpy.sqrt(2))*numpy.array([[1,-1],[1,1]])
        w,v=numpy.linalg.eigh(RotMat)
        givens=givensRotns(v.T)[0]
        circ=QuantumCircuit(2)
        for rots in givens:
            for tup in rots:
                circ.cnot(tup[1],tup[0])
                circ.cry(-2*tup[2],tup[0], tup[1])
                circ.cnot(tup[1], tup[0])
                circ.rz(tup[3],tup[1])        
        circ0_1gate=circ.to_instruction()
        return circ0_1gate,rotH
    def getCounts(Input):
        circ,num_shots=Input
        circ1=circ.copy()
        circ1.add_register(ClassicalRegister(8,'c'))
        circ1.measure([0,1,2,3,4,5,6,7],[0,1,2,3,4,5,6,7]) 
        job_sim = execute(circ1, AerSimulator(method='density_matrix'),shots=num_shots,basis_gates=basis_gates,noise_model=noise_model,had_transpiled=False) 
        result_sim = job_sim.result()
        #density_matrix = result_sim
        counts = result_sim.get_counts(circ1) 
        return counts#,density_matrix
    def estimate(PauliOp,counts):
        PauliString,coeff=PauliOp.primitive.to_list()[0]
        indexOfZs=numpy.where(numpy.array(list(PauliString))=='Z')[0]
        bitstrings,cnts=list(counts. keys()),list(counts.values())
        p=0
        count_valid=0
        for i in range(len(bitstrings)):
            bit_Str_to_Arr=numpy.array(list(bitstrings[i]))
            #flag1=1 if len(numpy.where(bit_Str_to_Arr=='1')[0])==4 else 0 #check number of electrons
            #flag2=1 if len(numpy.where(bit_Str_to_Arr[0:4]=='1')[0])==2 else 0 #check net spin
            #if ((flag1==1) and (flag2==1)):
            #count_valid=count_valid+cnts[i]
            counter_1=list(numpy.array(list(bitstrings[i]))[indexOfZs]).count('1')
            if counter_1%2==1:
                p=p+cnts[i]
        p=p/sum(cnts)
        mean=(1-2*p)
        return mean*coeff.real
    def meanOp(PauliOp,counts):
        mean=0
        for i in range(len(PauliOp)):
            mean=mean+estimate(PauliOp[i],counts)
        return mean 
    def covariance(Op,i,j,counts):
        cov=estimate(Op[i]@Op[j],counts)-estimate(Op[i],counts)*estimate(Op[j],counts)
        return cov
    def covbtnOps(A,B,counts):
        corr=0
        for i in range(len(A)):
            for j in range(len(B)):
                corr=corr+estimate(A[i]@B[j],counts)-estimate(A[i],counts)*estimate(B[j],counts)
        return corr        
    def variance(Op,counts):
        var=0
        cov_mat=numpy.zeros((len(Op),len(Op)))
        for i in range(len(Op)):
            for j in range(i,len(Op)):
                if i==j:
                    mean=estimate(Op[i],counts)
                    coeff=Op[i].primitive.to_list()[0][1].real
                    cov_mat[i][j]=(coeff**2-mean**2)
                    var=var+(coeff**2-mean**2)
                elif i!=j:
                    var=var+2*covariance(Op,i,j,counts)
                    cov_mat[i][j]=cov_mat[j][i]=covariance(Op,i,j,counts)
        return var
    def OpErr(data):
        Op,counts=data
        var=variance(Op,counts)
        err=numpy.sqrt(var/sum(counts.values()))
        return err.real
    U=7
    circ=var_form_base.construct_circuit(params)
    eg_h1_0,eg_h1_1,eg_h2_0,eg_h2_1=egBandHamiltonianPartition(U)
    qubitH_0,qubitH_1,qubitH_2=qubitHamiltonianGroups(U)
    opArr=[qubitH_0,qubitH_1,qubitH_2]
    #two body off diags
    circ0_1gate,rotH2Body=OffDiag2BodyTerms(opArr[2])
    #one body off diags
    circ0_8gate,rotH1Body=OffDiag1BodyTerms(opArr[0])
    Ops=[rotH1Body,opArr[1],rotH2Body]
    circ_arr=[circ.copy(),circ.copy(),circ.copy()]
    circ_arr[0].append(circ0_8gate,[0,1,2,3,4,5,6,7])
    circ_arr[2].append(circ0_1gate,[0,1])
    circ_arr[2].append(circ0_1gate,[4,5])
    counts_Arr=Parallel(n_jobs=3,verbose=2)(delayed(getCounts)((circ_arr[i],num_shots_arr[i])) for i in range(len(circ_arr)))
    m1=meanOp(Ops[0],counts_Arr[0])
    m2=meanOp(Ops[1],counts_Arr[1])
    m3=meanOp(Ops[2],counts_Arr[2])
    E=m1+m2+m3
    if (save_steps==True):
        with open('eg_model_BFGS_JW_shots.txt','+a') as f:
            Str=["{:0.16f}".format(params[i].real) for i in range(len(params))]
            print('['+','.join(Str)+']'+'#'+"{:0.16f}".format(E.real),file=f)
    err1=OpErr((Ops[0],counts_Arr[0]))
    err2=OpErr((Ops[1],counts_Arr[1]))
    err3=OpErr((Ops[2],counts_Arr[2]))
    return E,err1+err2+err3

In [6]:
#Pool construction
def commutator(A,B):
        B2=WeightedPauliOperator([[-B.paulis[0][0],B.paulis[0][1]]])
        return A.multiply(B).add(B2.multiply(A))
def commutatorsForGradient(Op):
    Op_conj=WeightedPauliOperator([[-1j,Op.paulis[0][1]]])
    M=Op.multiply(qubitH).add(qubitH.multiply(Op_conj))
    Mmat=op_converter.to_matrix_operator(M).matrix
    return Mmat
def commutatorPool(qubitH,stripZs=False):
    def stripZ(label):
        label=numpy.array(list(label))
        label[label=='Z']='I'
        label=''.join(list(label))
        return label
    def XYtrans(label):
        label=numpy.array(list(label))
        label1=copy.deepcopy(label)
        label1[label=='X']='Y'
        label1[label=='Y']='X'
        label1=''.join(list(label1))
        return label1
    #construct commutator pool from the Hamiltonian
    pool_H=[WeightedPauliOperator([[1j,qubitH.paulis[i][1]]]) for i in range(len(qubitH.paulis))][1:]
    #commutator between operators
    commutator_pool=[WeightedPauliOperator([[1j,commutator(op1,op2).paulis[0][1]]]) for op1 in pool_H for op2 in pool_H if countYgates(commutator(op1,op2).paulis[0][1].to_label())%2==1]
    labels=[commutator_pool[i].paulis[0][1].to_label() for i in range(len(commutator_pool))]
    unique_labels=numpy.unique(labels)
    #remove Z strings
    if stripZs==True:
        new_labels=[]
        for i in range(len(unique_labels)):
            new_labels.append(stripZ(unique_labels[i]))
        unique_labels=numpy.unique(new_labels)   
#     pool=[unique_labels[0]]
#     for i in range(1,len(unique_labels)):
#         if XYtrans(unique_labels[i]) in pool:
#             continue
#         else:
#             pool.append(unique_labels[i])
    commutator_pool=[WeightedPauliOperator([[1j,Pauli(unique_labels[i])]]) for i in range(len(unique_labels))]
    print(len(commutator_pool))
    return commutator_pool
def compute_gradient(M,state):
    grad=state@M@numpy.conjugate(state)
    return grad.real

In [7]:
#Optimizer
def SMO(cost,params,runs=20,tol=1e-4,save_opt_steps=False):
    index=1
    conv_err=1000
    def E_landscape(ind,ang,cost,params):
        params1=copy.deepcopy(params)
        params1[ind]=params1[ind]+ang #ang
        #circ=var_form_base.construct_circuit(parameters=params1)
        E=cost(params1)
        return E.real
    def determine_unknowns(E,cost,params,ind):
        L1=E#_landscape(ind,0,params,Hmat)
        L2=E_landscape(ind,numpy.pi/4.,cost,params)
        L3=E_landscape(ind,-numpy.pi/4.,cost,params)
        ratio=(L3-L2)/(2*L1-L2-L3)
        a3=(L2+L3)/2.
        a2=2*params[ind]-numpy.arctan(ratio)
        a1=(L1-a3)/numpy.cos(numpy.arctan(ratio))
        return a1,a2,a3
    def update(E,cost,params,ind):
        a1,a2,a3=determine_unknowns(E,cost,params,ind)
        thetaStar=a2/2.+numpy.pi/2. if a1>0 else a2/2.
        newParams=copy.deepcopy(params)
        newParams[ind]=thetaStar
        updEnergy=a3-a1 if a1>0 else a3+a1
        return newParams,updEnergy.real
    while conv_err>tol and index<runs:
        print("looped "+str(index)+" times")
        Eold=cost(params)
        init=0
        for i in range(len(params)):#[::-1][0:1]:
            #first run sequential minimial optimization (SMO)  for a given multiqubit operator using 
            #exact analytical form for cost function
            if init==0:
                E=Eold
            ind=i
            params,E=update(E,cost,params,ind)
            if save_opt_steps==True:
                with open('paramsForQubitAdapt_eg_model_JW.txt','a') as f:
                    Str=["{:0.16f}".format(params[i].real) for i in range(len(params))]
                    print('['+','.join(Str)+']'+'#'+"{:0.16f}".format(E.real),file=f)
            else:
                continue
            init=init+1  
        conv_err=Eold-E
        print("inner loop error",conv_err)
        index=index+1
    res={'fun':E,'x':params}    
    return res 

In [11]:
def AdaDelta_qasm(cost,params,runs=20,tol=1e-4,num_shots_arr=[2**8,2**9,2**9],save_opt_steps=False):
    string_shots_label=','.join([str(shots) for shots in num_shots_arr])
    v=numpy.zeros((len(params)))*0.0
    delta=numpy.zeros((len(params)))*0.0
    beta=0.9
    rho=0.8
    directions= numpy.arange(len(params))
    delta=0
    conv_err=1
    old_conv_err=1
    Eold=cost(params)
    E_arr=[]
    ind=0
    eps=numpy.array([5e-5]*len(params))#1e-5,5e-6,1e-7,5e-8
    Max=1e5
    lr0=0.02
    def learningRateEnv(lr0,runs):
        lr_u=lr0+Max*numpy.exp(-(35/runs)*numpy.linspace(0,runs,runs+1))
        lr_d=lr0*(1-numpy.exp(-(35/runs)*numpy.linspace(0,runs,runs+1)))
        return lr_u,lr_d
    def Energy(params):
        backend = Aer.get_backend('statevector_simulator')
        circ=var_form_base.construct_circuit(parameters=params)
        stateVector_0=execute(circ,backend,shots=1024).result().get_statevector()
        E=numpy.real(numpy.dot(numpy.dot(numpy.conjugate(stateVector_0),Hmat),stateVector_0)) 
        return E
    def StochGrad(cost,params,directions,num_shots_arr):
        shiftedParams1=[numpy.array(params)+numpy.array([0]*ei+[numpy.pi/4.]+[0]*(len(params)-ei-1)) for ei in directions]
        shiftedParams2=[numpy.array(params)+numpy.array([0]*ei+[-numpy.pi/4.]+[0]*(len(params)-ei-1)) for ei in directions]
        stochGrads=numpy.zeros((len(params)))
        estm1Arr=map(cost,shiftedParams1)
        estm2Arr=map(cost,shiftedParams2)
        stochGrads=0.5*(numpy.array(list(estm1Arr))-numpy.array(list(estm2Arr)))
        return stochGrads
    lr_u,lr_d= learningRateEnv(lr0,runs)
    flag=0
    flag1=0
    terminate=False
    while (conv_err>tol):
        print("index",ind)
        old_conv_err=conv_err
        g_stoch=StochGrad(cost,params,directions,num_shots_arr)
        v = beta*v+(1-beta)*g_stoch*g_stoch
        lr=numpy.sqrt(delta+eps)/numpy.sqrt(v+eps)  #learning rate
        #clipping learning rates, increasing shots and restarting ada delta
        if ((conv_err<1e-4) and (flag==0)):
            flag=1
            num_shots_arr=[2**16,2**16,2**16]
            print("************shots increased**************")
        if ((conv_err<4e-5) and (flag1==0)):
            flag1=1
            num_shots_arr=[2**18,2**20,2**20]
            print("************shots increased**************")    
        for i in range(len(lr)):
            if lr[i]>lr_u[ind]:
                eps[i]=lr_u[ind]*lr_u[ind]*v[i]
                lr[i]=lr_u[ind]
                delta[i]=0
            elif lr[i]<lr_d[ind]:
                eps[i]=lr_d[ind]*lr_d[ind]*v[i]
                lr[i]=lr_d[ind]
                delta[i]=0
            else:
                continue
        g=lr*g_stoch
        params=params-g
        params=params.real
        delta= beta*delta+(1-beta)*g*g
        E=Energy(params)#HamiltonianEstm(params)[0]
        E_arr.append(E)
        print("updated energy",E,"learning rate",lr)#,Energy(params))
        if save_opt_steps==True:
            with open('adaDeltaOptStepsWithQasm.txt','a') as f:
                print('['+','.join([str(params[i]) for i in range(len(params))])+']'+'#'+str(E),file=f)
        ind=ind+1 
        conv_err=rho*conv_err+(1-rho)*numpy.abs(Eold-E)
        print("error",conv_err)
        Eold=E
    res={'x':params,'fun':E}    
    return res

In [12]:
U=7
#Constructing Hamiltonian
qubitH,Hmat,Eg,state_g=egBandHamiltonian(U)
#commutator pools
commutator_pool=commutatorPool(qubitH,stripZs=True)
#constructing matrices for gradient computation
MatrixRepOfPoolOps=list(map(commutatorsForGradient,commutator_pool))
print("pool sizes",len(MatrixRepOfPoolOps))
backend=Aer.get_backend('statevector_simulator')
#preparing HF state
circ=QuantumCircuit(8)
circ.x(0)
circ.x(1)
circ.x(4)
circ.x(5)
var_form_base=UCCSD(8,num_particles=4, initial_state=circ,qubit_mapping='jordan_wigner',two_qubit_reduction=False)
var_form_base.manage_hopping_operators()
error=1000

32
pool sizes 32


$|\Psi\rangle =\exp(i\theta_{1}A_{1})\exp(i\theta_{2}A_{2})\ldots \exp(i\theta_{n}A_{n})|0\rangle$

In [67]:
# ExcOps=[]
# params=[]
# count=0
# with open('data_eg_model_JW.txt','r') as f:
#     lines=f.readlines()
#     for i in range(len(lines)):
#         line=lines[i]
#         if ((line[0]=='I') or (line[0]=='X') or (line[0]=='Y') or (line[0]=='Z')):
#             count=count+1
#             ExcOps.append(line[0:-1])
#             params=eval(lines[i+1])
#             var_form_base.push_hopping_operator(WeightedPauliOperator([[1j,Pauli(ExcOps[-1])]]))        
#         if count>32:
#             break

In [68]:
# old_circ=var_form_base.construct_circuit(parameters=params)
# old_circ=qiskit.transpile(old_circ,basis_gates=['u1','u2','u3','cx'])

In [69]:
#qasm_str=old_circ.qasm()

In [70]:
#circuit = zx.Circuit.load("circuit_chk.qasm")

In [71]:
#print(circuit.to_basic_gates().stats())

In [72]:
#g = circuit.to_graph() # We first have to convert the circuit into a ZX-graph

In [73]:
# display(zx.draw(g))
# print("Orig T-count:", zx.tcount(g))

In [74]:
# zx.simplify.full_reduce(g,quiet=True)
# print("The optimized ZX-diagram:")
# display(zx.draw(g))
# print("Optim T-count:", zx.tcount(g))

In [75]:
# new_circ = zx.extract_circuit(g)
#optimized_circ = zx.optimize.basic_optimization(new_circ.to_basic_gates())

In [76]:
# zx.draw(new_circ)

In [77]:

# circuit.verify_equality(optimized_circ)

In [78]:
# print(optimized_circ.stats())

In [79]:
circ_qasm=new_circ.to_qasm()

NameError: name 'new_circ' is not defined

In [588]:
circ=QuantumCircuit.from_qasm_str(circ_qasm)

In [589]:
circ.count_ops()

OrderedDict([('cz', 112), ('cx', 55), ('h', 54), ('rz', 47)])

In [566]:
old_circ.count_ops()

OrderedDict([('cx', 152), ('u2', 80), ('u1', 33)])

In [20]:
def NoiseModel(p_meas=0.00,prob1=0.01,prob2=0.007):
    # Error probabilities
    #measurement error
    error_meas = noise.pauli_error([('X',p_meas), ('I', 1 - p_meas)])
    #prob_1 - 1-qubit gate
    #prob_2 - 2-qubit gate
    # Depolarizing quantum errors
    error_1 = noise.depolarizing_error(prob1, 1)
    error_2 = noise.depolarizing_error(prob2, 2)
    # Add errors to noise model
    noise_model = noise.NoiseModel()
    noise_model.add_all_qubit_quantum_error(error_meas, "measure")
    noise_model.add_all_qubit_quantum_error(error_1, ['u1', 'u2', 'u3'])
    noise_model.add_all_qubit_quantum_error(error_2, ['cx'])
    # Get basis gates from noise model
    basis_gates = noise_model.basis_gates
    return noise_model,basis_gates
noise_model,basis_gates=NoiseModel()#(p_meas=0.0,prob1=0.0001,prob2=0.0001)

In [33]:
Energy_Estimator(old_circ,num_shots_arr=[2**14,2**16,2**16],noise_model=noise_model,basis_gates=basis_gates)

NameError: name 'old_circ' is not defined

In [590]:
circ.depth(),old_circ.depth()

(196, 192)

In [34]:
def fun_jac(params):
    circ=var_form_base.construct_circuit(parameters=params)
    state=execute(circ,backend,shots=1024).result().get_statevector() 
    def State(params):
        circ=var_form_base.construct_circuit(parameters=params)
        state=execute(circ,backend,shots=1024).result().get_statevector() 
        #E=(numpy.conjugate(state)@Hmat@state).real
        return state
    setofParams1=[numpy.array(params)+numpy.array([0]*i+[numpy.pi/2]+[0]*(len(params)-i-1)) for i in range(len(params))]
    States=numpy.array(Parallel(n_jobs=7,verbose=2)(delayed(State)(setofParams1[i]) for i in range(len(params))))#qubit_pool_UCCSD))))#
    jac=[]
    for i in range(len(States)):
        jac.append((numpy.conjugate(state)@Hmat@States[i]).imag)
    return jac

In [20]:
# save_steps=True
# res=scipy.optimize.minimize(Energy,params,method='BFGS',options={'ftol':1e-7})
# params,E=list(res['x']),res['fun']

In [13]:
params=[]
EnergyArr=[]
ExcOps=[]
ti=time.time()
steps=35
for i in range(35):
    save_steps=False
    circ=var_form_base.construct_circuit(parameters=params)
    state=execute(circ,backend,shots=1024).result().get_statevector()
    grads=numpy.array(Parallel(n_jobs=7,verbose=2)(delayed(compute_gradient)(MatrixRepOfPoolOps[i],state) for i in range(len(commutator_pool))))#qubit_pool_UCCSD))))#
    indexes=numpy.argsort(abs(grads))[::-1][:3]
    print("three highest grads",grads[indexes[0]],grads[indexes[1]],grads[indexes[2]])
    if len(ExcOps)!=0:
        if (commutator_pool[indexes[0]].paulis[0][1].to_label()!=ExcOps[-1]):
            PauliOp=commutator_pool[indexes[0]]
        else:
            PauliOp=commutator_pool[indexes[1]]
    else:
        PauliOp=commutator_pool[indexes[0]]
    ExcOps.append(PauliOp.paulis[0][1].to_label())
    print("chosen Op",ExcOps[-1])
    with open('adaDeltaOptStepsWithQasm.txt','a') as f:
        print("label-",ExcOps[-1],file=f)
    params.append(0.0)
    var_form_base.push_hopping_operator(PauliOp)
    #construct circuit for estimating Hamiltonian
#     circ=var_form_base.construct_circuit(parameters=params)
    res=AdaDelta_qasm(Energy_Estimator,params,runs=150,tol=1e-5,num_shots_arr=[2**11,2**12,2**12],save_opt_steps=True)
    E,params=res['fun'],list(res['x'])
    save_steps=True
    #res=scipy.optimize.minimize(Energy_Estimator,params,method='BFGS',options={'ftol':1e-7})
    #params,E=list(res['x']),res['fun']
    print("num_params",var_form_base.num_parameters)
    print("Energy",E)
    EnergyArr.append(E)
    print("time elapsed",time.time()-ti)    
    error=EnergyArr[-1]-Eg

[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  19 out of  32 | elapsed:    1.0s remaining:    0.7s
[Parallel(n_jobs=7)]: Done  32 out of  32 | elapsed:    1.0s finished


three highest grads -0.603251884 0.603251884 0.603251884
chosen Op YIXIIIII


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    3.6s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


index 0


[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.5s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.4s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


...

updated energy -18.863546544622643 learning rate [0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02]
error 8.50156282962116e-05
index 150


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.7s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.7s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.7s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.7s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.7s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.8s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:   

updated energy -18.86361019479963 learning rate [0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02]
error 8.074253803433735e-05
index 151


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.6s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.7s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.7s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.7s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.7s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.8s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:   

IndexError: index 151 is out of bounds for axis 0 with size 151

In [19]:
params_arr=[]
Labels_arr=[]
Energy_arr=[]
count=0
with open('paramsForQubitAdapt_eg_model_JW.txt','r') as f:
    lines=f.readlines()
    for i in range(len(lines)):
        if lines[i][0]=='l':
            count=count+1
            label=lines[i].split('-')[1][1:-1]
            Labels_arr.append(label)
            if count>1:
                params,E=lines[i-1].split('#')
                params_arr.append(eval(params))
                Energy_arr.append(float(E))
    params_arr.append(eval(lines[-1]))
    Energy_arr.append(float(lines[-1].split('#')[1]))

In [22]:
params_arr=params_arr[0:33]
Energy_arr=Energy_arr[0:33]

In [23]:
for i in range(len(params_arr)):
    with open('data_eg_model_JW.txt','+a') as f:
            print(Labels_arr[i]+'\n'+'['+','.join([str(params_arr[i][j]) for j in range(len(params_arr[i]))])+']'+'#'+"{:0.12f}".format(Energy_arr[i]),file=f)

In [2]:
0.0003, 3e-4

(0.0003, 0.0003)